In [ ]:
import imaplib
import email
import yaml

import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
from email.header import decode_header

import numpy as np
from PIL import Image
from wordcloud import WordCloud, STOPWORDS
from matplotlib.colors import LinearSegmentedColormap

In [ ]:
with open('/content/drive/MyDrive/Resources/credenciales.yaml') as f:
    content = f.read()
my_credentials = yaml.load(content, Loader = yaml.FullLoader)
user, password = my_credentials['user'], my_credentials['password']

imap_url ='imap.gmail.com'
my_mail = imaplib.IMAP4_SSL(imap_url)
my_mail.login(user, password)

error: ignored

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:

my_mail.select('Inbox')

error: ignored

In [ ]:
total_correos = int(my_mail.select('Inbox')[1][0].decode('utf-8'))
total_correos

In [ ]:
email_df = pd.DataFrame(columns=['Date','From','Subject'], index=range(total_correos,0,-1))

for i in tqdm(range(total_correos, 0, -1)):
    data = my_mail.fetch(str(i),'(UID RFC822)')
    for response_part in data:
        arr = response_part[0]
        if isinstance(arr, tuple):
            try: msg = email.message_from_string(str(arr[1],'utf-8'))
            except: msg = email.message_from_string(str(arr[1],'latin-1'))
            new_row = pd.Series({"Date":msg['Date'] , "From":msg['from'], "Subject":msg['subject']})
    email_df = email_df.append(new_row, ignore_index=True)

email_df = email_df.dropna(how='all').reset_index(drop=True)
email_df

In [ ]:
email_df.to_csv('Data/email_df.csv', index=False, encoding='utf-8')

In [ ]:
# email_df = pd.read_csv('Data/email_df.csv').fillna('')
# email_df.head(20)

In [ ]:
# print(msg.keys())

In [ ]:
def get_email_from(x):
    x_mod = x.split('<')[-1].split('>')[0]
    return x_mod


In [ ]:
from email.header import decode_header

def get_name_from(x):
    text, encoding = decode_header(x)[0]
    if not encoding and isinstance(text, str):
        text = ' '.join(text.split(' ')[:-1])
    else:
        text = text.decode('utf-8', errors='ignore')

    text = text.replace('"', '')
    return text


In [ ]:
from email.header import decode_header

def clean_subject(x):
    if x:
        text, encoding = decode_header(x)[0]
        try:
            text = text.decode('utf-8', errors='ignore')
        except:
            pass
    else:
        text = x
    return text


In [ ]:
# Extract email addresses from the 'From' column and create a new 'Mail' column
email_df['Mail'] = email_df['From'].apply(lambda x: get_email_from(x))

# Extract names from the 'From' column and create a new 'Name' column
email_df['Name'] = email_df['From'].apply(lambda x: get_name_from(x))

# Clean and decode subject lines from the 'Subject' column and update it
email_df['Subject'] = email_df['Subject'].apply(lambda x: clean_subject(x))

# Drop the 'From' column from the DataFrame
email_df = email_df.drop(columns=['From'])

# Display the first few rows of the modified DataFrame
email_df.head()


In [ ]:
def clean_date(x):
    if ',' not in x:
        x = ', ' + x
    if '(' in x:
        x = ' '.join(x.split(' ')[:-1])
    x = ' '.join(x.split(' ')[:-1])
    return x


In [ ]:
def obtener_dia(x):
    x = x.strftime('%A')
    return x

In [ ]:
# Clean and format the 'Date' column
email_df['Date'] = email_df['Date'].apply(lambda x: clean_date(x))

# Split the cleaned date string by ', ' and take the last part
email_df['Date'] = email_df['Date'].str.split(', ').str[-1]

# Extract 'H_M_S' (Hour, Minute, Second) part from the 'Date' column
email_df['H_M_S'] = email_df['Date'].apply(lambda x: x[-8:])

# Calculate 'Hour' column based on 'H_M_S'
email_df['Hour'] = email_df['H_M_S'].apply(lambda x: x[:2]+'h-'+str(int(x[:2])+1).zfill(2)+'h')

# Modify 'Date' column to have the day, month, and year format
email_df['Date'] = email_df['Date'].apply(lambda x: x[:-9] if len(x[:-9])==11 else '0'+x[:-9])

# Convert the 'Date' column to datetime format using the specified format
email_df['Date'] = pd.to_datetime(email_df['Date'], format='%d %b %Y')

# Obtain the 'WeekDay' column
email_df['WeekDay'] = email_df['Date'].apply(lambda x: get_day(x))

# Display the first few rows of the modified DataFrame
email_df.head()


In [ ]:
email_df.to_csv('Data/email_df_final.csv', quotechar='"', sep='|', index=False, encoding='utf-8')

In [ ]:
import numpy as np
from matplotlib.colors import LinearSegmentedColormap

# Define a function to interpolate values from the range [0, 255] to the range [0, 1]
def inter_from_256(x):
    return np.interp(x=x, xp=[0, 255], fp=[0, 1])

# Define color dictionary using interpolation for the 'red', 'green', and 'blue' channels
cdict = {
    'red': (
        (0.0, inter_from_256(197), inter_from_256(197)),
        (1/5*1, inter_from_256(52), inter_from_256(52)),
        (1/5*3, inter_from_256(66), inter_from_256(66)),
        (1/5*4, inter_from_256(234), inter_from_256(234)),
        (1.0, inter_from_256(251), inter_from_256(251))
    ),
    'green': (
        (0.0, inter_from_256(34), inter_from_256(34)),
        (1/5*1, inter_from_256(168), inter_from_256(168)),
        (1/5*3, inter_from_256(133), inter_from_256(133)),
        (1/5*4, inter_from_256(67), inter_from_256(67)),
        (1.0, inter_from_256(188), inter_from_256(188))
    ),
    'blue': (
        (0.0, inter_from_256(31), inter_from_256(31)),
        (1/5*1, inter_from_256(83), inter_from_256(83)),
        (1/5*3, inter_from_256(244), inter_from_256(244)),
        (1/5*4, inter_from_256(53), inter_from_256(53)),
        (1.0, inter_from_256(4), inter_from_256(4))
    ),
}

# Create a LinearSegmentedColormap named 'gmail' using the defined color dictionary
gmail = LinearSegmentedColormap('gmail', segmentdata=cdict)


In [ ]:
from wordcloud import WordCloud, STOPWORDS
import numpy as np
from PIL import Image

# Initialize an empty string to store subject words
total_subjects = ' '

# Update the set of stopwords with additional words
stopwords = STOPWORDS.update(['al', 'de', 'en', ...])  # A list of common stopwords

# Load an image to use as a mask for the word cloud
mask = np.array(Image.open('Resources/gmail.jpg'))

# Iterate through the subjects in the DataFrame and concatenate subject words
for mensaje in email_df['Subject'].values:
    subjects = str(mensaje).lower().split()
    for subject in subjects:
        total_subjects = total_subjects + subject + ' '

# Create a WordCloud instance with custom settings
wordcloud = WordCloud(
    scale=2.5,
    width=1000,
    height=800,
    background_color='black',
    mask=mask,
    colormap=gmail,  # Using the custom colormap named 'gmail'
    stopwords=stopwords,
    min_font_size=6
).generate(total_subjects)

# Convert the word cloud to an image
wordcloud.to_image()
